# Example of modelling: TOI-270

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (10,5)

To define a star:

In [ ]:
from ravest.model import Star

star = Star(name="TOI-270", mass=0.386)
print(star)

Let's define some planets. These parameter values for TOI-270 b, c, and d were obtained from [Van Eylen et al. 2021](https://doi.org/10.1093/mnras/stab2143). The full list of parameterisations supported can be seen at `ravest.basis.BASIS_STR`.

In [ ]:
from ravest.model import Planet
from ravest.basis import Basis

basis = Basis("per k e w tc")

planet_b = Planet(letter="b", basis=basis, params={"per": 3.3601538, "k": 1.27, "e": 0.034, "w": 0,   "tc": 2458387.09505})
planet_c = Planet(letter="c", basis=basis, params={"per": 5.6605731, "k": 4.16, "e": 0.027, "w": 0.2, "tc": 2458389.50285})
planet_d = Planet(letter="d", basis=basis, params={"per": 11.379573, "k": 2.56, "e": 0.032, "w":-0.1, "tc": 2458389.68186})

We can see what the radial velocity would be due to the effect of one of the planets:

In [ ]:
times = np.linspace(2458774, 2458874, 1000)

plt.figure()
plt.title(planet_d)
plt.xlabel("Time")
plt.ylabel("Radial velocity [m/s]")
plt.plot(times, planet_d.radial_velocity(times))

Let's add the planets to the star, and use that to model the combined radial velocity that we would see, due to the effect of all of the planets. Notice that we also define a `Trend` object. This can store a constant velocity offset $\gamma$ ($\rm{ms}^{-1}$), a linear trend $\dot{\gamma}$, ($\rm{ms}^{-1}/\rm{day}$), and a quadratic trend $\ddot{\gamma}$ ($\rm{ms}^{-1}/\rm{day}^2$) which are often instrumental contributions, or possibly undetected long-period companions.

In [ ]:
star.add_planet(planet_b)
star.add_planet(planet_c)
star.add_planet(planet_d)

from ravest.model import Trend
star.add_trend(Trend(params={"g":0, "gd":0, "gdd":0}))

plt.figure()
plt.title(star)
plt.xlabel("Time")
plt.ylabel("Radial velocity [m/s]")
plt.plot(times, star.radial_velocity(times))

If we had some observed data, we can plot the data on top of the system RV model, and look at the residuals and generate phase plots for each planet. As a quick example, I've generated some example data for the TOI-270 system.

In [ ]:
ti, rv, err = np.loadtxt("example_data/TOI270.txt")
star.phase_plot(ti, rv, err)